In [1]:
from transformers import BertTokenizer, BertForTokenClassification, pipeline, get_linear_schedule_with_warmup
from transformers import CanineTokenizer, CanineForTokenClassification
from seqeval.metrics import classification_report
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd
import torch
import os
import re


In [2]:
CANINE = True


pth = '/home/sondors/Documents/price/BERT_NER/csv/NER_2609.csv'
# pth = '/home/sondors/Documents/price/Разметка для Игоря Bridgestone.xlsx'
train_batch_size = 2
val_batch_size = 2
device = 'cuda'

label2id = {'B-width': 1,
        'B-height': 2,
        'B-radius': 3,
        'B-brand': 4,
        'B-line': 5,
        'O': 0}

id2label = {1: 'B-width',
        2: 'B-height', 
        3: 'B-radius', 
        4: 'B-brand', 
        5: 'B-line', 
        0: 'O'}

if CANINE:
        from custom_train_loop_CANINE import load_model, load_data, valid

        model_pth = "/home/sondors/Documents/price/BERT_NER/weights/CANINE/epoch_4"
        model = CanineForTokenClassification.from_pretrained(model_pth).to(device)
        tokenizer = CanineTokenizer.from_pretrained(model_pth)
        max_seq_length = 2048
        train_size = 0.9
else:
        from custom_train_loop import load_model, load_data, valid

        # model_pth = "/home/sondors/29887"
        model_pth = '/home/sondors/Documents/price/BERT_NER/weights/BERT/rubert_tiny_1/epoch_5'
        tokenizer, model = load_model(model_pth, device, label2id, id2label)
        max_seq_length = 512
        train_size = 0.8


Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [3]:
training_loader, testing_loader = load_data(tokenizer, pth, train_size, label2id, train_batch_size, val_batch_size, max_seq_length)

FULL Dataset: (7018, 8)
TRAIN Dataset: (6316, 8)
TEST Dataset: (702, 8)


In [4]:
report = valid(model, testing_loader, id2label, device)
print(f"{report}\n")

Validation loss per 100 evaluation steps: 1.3045773812336847e-05
Validation loss per 100 evaluation steps: 0.00014247153378146502
Validation loss per 100 evaluation steps: 0.00015686696808451012
Validation loss per 100 evaluation steps: 0.00020045712743169822
Validation Loss: 0.0001792172312171557
Validation Accuracy: 0.997150253388677
              precision    recall  f1-score   support

       brand       1.00      1.00      1.00      4609
      height       1.00      1.00      1.00      1224
        line       0.94      0.99      0.96       599
      radius       1.00      1.00      1.00      1451
       width       1.00      1.00      1.00      2097

   micro avg       0.99      1.00      1.00      9980
   macro avg       0.99      1.00      0.99      9980
weighted avg       0.99      1.00      1.00      9980




In [4]:
report = valid(model, testing_loader, id2label, device)
print(f"{report}\n")

Validation loss per 100 evaluation steps: 3.178526822011918e-05
Validation loss per 100 evaluation steps: 3.5747187320168344e-05
Validation loss per 100 evaluation steps: 0.00017520954822172296
Validation loss per 100 evaluation steps: 0.00020040577726414567
Validation Loss: 0.00017921726102837092
Validation Accuracy: 0.9970867887629979
              precision    recall  f1-score   support

       brand       1.00      1.00      1.00      4609
      height       1.00      1.00      1.00      1224
        line       0.94      0.99      0.96       599
      radius       1.00      1.00      1.00      1451
       width       1.00      1.00      1.00      2097

   micro avg       0.99      1.00      1.00      9980
   macro avg       0.99      1.00      0.99      9980
weighted avg       0.99      1.00      1.00      9980




In [5]:
def price_data_raw():
    pth = "/home/sondors/Documents/price/BERT_NER/csv/list1.xlsx"
    df = pd.read_excel(pth)

    clms = ['PRICE_FRNO', 
        'PRICE_FNAM', 
        'PRICE_CAID', 
        'PRICE_CNAM',  
        'PRICE_TMNO', 
        'PRICE_PRCN', 
        'PRICE_VERT', 
        'PRICE_PCOD', 
        'date', 
        'PRICE_IMGN', 
        'PRICE_LOCD', 
        'lineNumber',
        'PRICE_PRDS',
        'PRICE_DPRN',
        'PRICE_PTYP',
        'PRICE_PROMO',
        'PRICE_CURR',
        'PRICE_LOCA',
        'PRICE_WARR',
        'PRICE_RSLT',
        'PRICE_PLID',
        'PRICE_SALES',
        'PRICE_DTSH',
        'PRICE_BARCODE',
        'комментарий',
        'kpl',
        'PRICE_PARAM',
        'PRICE_NOTE']

    clms_hz = ['kpl',
        'PRICE_NOTE',
        'PRICE_PARAM']

    df = df.drop(columns=clms + clms_hz)
    return df
df = price_data_raw()
df

,PRICE_NAME,Ширина,Высота,Радиус,Инд. Скорост,XL,RFT,Линейка,ppl
0,"10,5/31 R15 Bridgestone Dueler A/T 001 109S",NaN,NaN,NaN,109S,NaN,NaN,Dueler,Bridgestone
1,12067 Bridgestone Автошина R20 275/50 Bridgest...,275,50,20,113R,XL,NaN,Blizzak DM-V2,Bridgestone
2,12088 BRIDGESTONE Автошина R20 255/50 Bridgest...,255,50,20,109T,XL,NaN,Blizzak DM-V2,Bridgestone
3,13/175/70 Bridgestone Ice Cruiser 7000S 82T ш,175,70,17,82S,NaN,NaN,Ice Cruiser 7000S,Bridgestone
4,13602 BRIDGESTONE Автошина R18 275/35 Bridgest...,275,35,18,95S,NaN,NaN,Blizzak Ice,Bridgestone
...,...,...,...,...,...,...,...,...,...
23143,"Бриджстоун 2657015 T 112 AT001, BRIDGESTONE 12...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bridgestone
23144,"Бриджстоун 2753518 S 95 VRX, BRIDGESTONE 8393 ...",NaN,NaN,NaN,NaN,NaN,NaN,Blizzak VRX,Bridgestone
23145,Бриджстоун 315/70r22.5 R249 Tl 152/154 L/M Рул...,315,70,15,152/154,NaN,NaN,NaN,BRIDGESTONE
23146,шинаBridgestone LT245/75R16 120Q Dueler M/T 674,245,75,16,120Q,NaN,NaN,Dueler,Bridgestone


In [6]:
offer_1 = df.iloc[23145]['PRICE_NAME']
offer_1

'Бриджстоун 315/70r22.5 R249 Tl 152/154 L/M Рулевая Bridgestone 12646'

In [7]:
df.iloc[23145]

PRICE_NAME      Бриджстоун 315/70r22.5 R249 Tl 152/154 L/M Рул...
Ширина                                                        315
Высота                                                         70
Радиус                                                         15
Инд. Скорост                                              152/154
XL                                                            NaN
RFT                                                           NaN
Линейка                                                       NaN
ppl                                                   BRIDGESTONE
Name: 23145, dtype: object

In [8]:
def inference(offer, model, tokenizer):
    pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
    return pipe(offer)

def brand_line_width_height_radius(result):
    # Создаем словари для каждой сущности
    entities = {}
    for item in result:
        entity_group = item['entity_group']
        word = item['word']
        score = item['score']
        if score > 0.7:
            if entity_group in entities:
                entities[entity_group].append(word)
            else:
                entities[entity_group] = [word]

    # Формируем строки для каждой сущности
    brand_str = ''.join(entities.get('brand', []))
    width_str = ''.join(entities.get('width', []))
    height_str = ''.join(entities.get('height', []))
    radius_str = ''.join(entities.get('radius', []))
    line_str = ''.join(entities.get('line', []))

    # Выводим строки
    print(f"Brand: {brand_str}")
    print(f"Line: {line_str}")
    print(f"Width: {width_str}")
    print(f"Height: {height_str}")
    print(f"Radius: {radius_str}")
    return brand_str, line_str, width_str, height_str, radius_str

def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    pattern1 = r'([rR])(\d)'
    processed_text = re.sub(pattern1, r'R \2', input_text)

    pattern2 = r'(\d)([rR])'
    processed_text = re.sub(pattern2, r'\1 R', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")

    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text


result = inference(offer_1, model, tokenizer)
print(f"result: {result}")
print(f"offer_1: {offer_1}\n")
brand_line_width_height_radius(result)
print("\nПосле препроцессинга:\n")
result = inference(process_text(offer_1), model, tokenizer)
print(f"result: {result}")
brand_line_width_height_radius(result)

result: [{'entity_group': 'brand', 'score': 0.9991767, 'word': 'Б', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99918956, 'word': 'р', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99927276, 'word': 'и', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.9993161, 'word': 'д', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.9991872, 'word': 'ж', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99908566, 'word': 'с', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99919134, 'word': 'т', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99925166, 'word': 'о', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.9987607, 'word': 'у', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.9991509, 'word': 'н', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.96527493, 'word': ' ', 'start': None, 'end': None}, {'entity_group': 

('Бриджстоун', '', '3152', '', '2')

In [12]:
result = inference(offer_1, model, tokenizer)
print(f"result: {result}")
print(f"offer_1: {offer_1}\n")
brand_line_width_height_radius(result)
print("\nПосле препроцессинга:\n")
result = inference(process_text(offer_1), model, tokenizer)
print(f"result: {result}")
brand_line_width_height_radius(result)

result: [{'entity_group': 'brand', 'score': 0.9945373, 'word': 'Брид', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99448264, 'word': '##ж', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99449366, 'word': '##сто', 'start': None, 'end': None}, {'entity_group': 'brand', 'score': 0.99435174, 'word': '##ун', 'start': None, 'end': None}, {'entity_group': 'width', 'score': 0.9854799, 'word': '315', 'start': None, 'end': None}, {'entity_group': 'height', 'score': 0.98213804, 'word': '70', 'start': None, 'end': None}, {'entity_group': 'line', 'score': 0.8588647, 'word': '##r', 'start': None, 'end': None}, {'entity_group': 'line', 'score': 0.896467, 'word': '##22', 'start': None, 'end': None}, {'entity_group': 'radius', 'score': 0.9158097, 'word': '.', 'start': None, 'end': None}, {'entity_group': 'radius', 'score': 0.88272935, 'word': '5', 'start': None, 'end': None}, {'entity_group': 'line', 'score': 0.5341704, 'word': 'R2', 'start': None, 'end': None}

('Брид##ж##сто##ун', '', '315', '70', '225')